#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
## PEC3: Noviembre 2018
#Practica sobre cómo generar un flujo de ejecución en un problema de Machine Learning
## UOC - Máster en Data Science
### Alumno: **Fernando Antonio Barbeiro Campos** - fbarbeiro@uoc.edu

Esta práctica simula un ejercicio completo de ETL (Extract-Transform-Load) junto a un análisis exploratorio de un dataset real, para posteriormente aplicar differentes algoritmos de aprendizaje automático que resuelvan un problema de regresión.

** This notebook covers: **
* *Parte 1: Conocimiento del dominio*
* *Parte 2: Extracción, transformación y carga [ETL] del dataset* (1 punto sobre 10)
* *Parte 3: Explorar los datos* (1 puntos sobre 10)
* *Parte 4: Visualizar los datos* (1 puntos sobre 10)
* *Parte 5: Preparar los datos* (1 puntos sobre 10)
* *Parte 6: Modelar los datos* (2 puntos sobre 10)
* *Parte 7: Ajustar y evaluar* (2 puntos sobre 10)
* *Parte 8: Propuesta de modelo altenativo* (2 puntos sobre 10)

*Nuestro objetivo será predecir de la forma más exacta posible la energía generada por un conjunto de plantas eléctricas usando los datos generados por un conjunto de sensores.*


## Parte 1: Conocimiento del dominio

** Background **

La generación de energía es un proceso complejo, comprenderlo para poder predecir la potencia de salida es un elemento vital en la gestión de una planta energética y su conexión a la red. Los operadores de una red eléctrica regional crean predicciones de la demanda de energía en base a la información histórica y los factores ambientales (por ejemplo, la temperatura). Luego comparan las predicciones con los recursos disponibles (por ejemplo, plantas, carbón, gas natural, nuclear, solar, eólica, hidráulica, etc). Las tecnologías de generación de energía, como la solar o la eólica, dependen en gran medida de las condiciones ambientales, pero todas las centrales eléctricas son objeto de mantenimientos tanto planificados y como puntuales debidos a un problema.

En esta practica usaremos un ejemplo del mundo real sobre la demanda prevista (en dos escalas de tiempo), la demanda real, y los recursos disponibles de la red electrica de California: http://www.caiso.com/Pages/TodaysOutlook.aspx

![](http://content.caiso.com/outlook/SP/ems_small.gif)

El reto para un operador de red de energía es cómo manejar un déficit de recursos disponibles frente a la demanda real. Hay tres posibles soluciones a un déficit de energía: construir más plantas de energía base (este proceso puede costar muchos anos de planificación y construcción), comprar e importar de otras redes eléctricas regionales energía sobrante (esta opción puede ser muy cara y está limitado por las interconexiones entre las redes de transmisión de energía y el exceso de potencia disponible de otras redes), o activar pequeñas [plantas de pico](https://en.wikipedia.org/wiki/Peaking_power_plant). Debido a que los operadores de red necesitan responder con rapidez a un déficit de energía para evitar un corte del suministro, estos basan sus decisiones en una combinación de las dos últimas opciones. En esta práctica, nos centraremos en la última elección.

** La lógica de negocio **

Debido a que la demanda de energía solo supera a la oferta ocasionalmente, la potencia suministrada por una planta de energía pico tiene un precio mucho más alto por kilovatio hora que la energía generada por las centrales eléctricas base de una red eléctrica. Una planta pico puede operar muchas horas al día, o solo unas pocas horas al año, dependiendo de la condición de la red eléctrica de la región. Debido al alto coste de la construcción de una planta de energía eficiente, si una planta pico solo va a funcionar por un tiempo corto o muy variable, no tiene sentido económico para que sea tan eficiente como una planta de energía base. Además, el equipo y los combustibles utilizados en las plantas base a menudo no son adecuados para uso en plantas de pico.

La salida de potencia de una central eléctrica pico varía dependiendo de las condiciones ambientales, por lo que el problema de negocio a resolver se podría describir como _predecir la salida de potencia de una central eléctrica pico en función de la condiciones ambientales_  - ya que esto permitiría al operador de la red hacer compensaciones económicas sobre el número de plantas pico que ha de conectar en cada momento (o si por el contrario le interesa comprar energía más cara de otra red).

Una vez descrita esta lógica de negocio, primero debemos proceder a realizar un análisis exploratorio previo y trasladar el problema de negocio (predecir la potencia de salida en función de las condiciones medio ambientales) en un tarea de aprendizaje automático (ML). Por ejemplo, una tarea de ML que podríamos aplicar a este problema es la regresión, ya que tenemos un variable objetivo (dependiente) que es numérica. Para esto usaremos [Apache Spark ML Pipeline](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark-ml-package) para calcular dicha regresión.

Los datos del mundo real que usaremos en esta práctica se componen de 9.568 puntos de datos, cada uno con 4 atributos ambientales recogidos en una Central de Ciclo Combinado de más de 6 años (2006-2011), proporcionado por la Universidad de California, Irvine en [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)). Para más detalles sobre el conjunto de datos visitar la página de la UCI, o las siguientes referencias:

* Pinar Tufekci, [Prediction of full load electrical power output of a base load operated combined cycle power plant using machine learning methods](http://www.journals.elsevier.com/international-journal-of-electrical-power-and-energy-systems/), International Journal of Electrical Power & Energy Systems, Volume 60, September 2014, Pages 126-140, ISSN 0142-0615.
* Heysem Kaya, Pinar Tufekci and Fikret S. Gurgen: [Local and Global Learning Methods for Predicting Power of a Combined Gas & Steam Turbine](http://www.cmpe.boun.edu.tr/~kaya/kaya2012gasturbine.pdf), Proceedings of the International Conference on Emerging Trends in Computer and Electronics Engineering ICETCEE 2012, pp. 13-18 (Mar. 2012, Dubai).

**Tarea a realizar durante la primera parte:**

Revisar la documentacion y referencias de:
* [Spark Machine Learning Pipeline](https://spark.apache.org/docs/latest/ml-guide.html#main-concepts-in-pipelines).
* [Databricks File System](https://docs.databricks.com/user-guide/dbutils.html).

## Parte 2: Extracción, transformación y carga [ETL] del dataset


Ahora que entendemos lo que estamos tratando de hacer, el primer paso consiste en cargar los datos en un formato que podemos consultar y utilizar fácilmente. Esto se conoce como ETL o "extracción, transformación y carga". Primero, vamos a cargar nuestro archivo de Amazon S3.

Nota: Como alternativa podemos subir nuestros datos utilizando "Databricks Menu> Tablas> Crear tabla", suponiendo que tengamos los archivos sin procesar en nuestro ordenador local.

Nuestros datos están disponibles en Amazon S3 en la siguiente ruta:

```
dbfs:/databricks-datasets/power-plant/data
```

==========================================================================================================================================================================================================
### Ejercicio 1 

Empezaremos por visualizar una muestra de los datos. Para esto usaremos las funciones pre-definidas en los notebooks de Databricks para explorar su sistema de archivos. Estas utilidades son las llamadas `dbutils` para trabajar en el llamado Databricks File System. La función `dbutils.fs.ls` permite listar los contenidos de un directorio. 

`dbutils.fs` dispone de su propio help, esta ayuda nos será de gran utilidad cuando deseemos ver las diferentes funciones disponibles.

In [5]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

Usar la función `display` y la función `dbutils.fs.ls` de Databricks para listar los ficheros del directorio S3 en el que se encuentran nuestros datos que usaremos en esta PEC.

In [7]:
#TODO: use display to list all the files of the directory containing the data
display(dbutils.fs.ls("dbfs:/databricks-datasets/power-plant/data"))

path,name,size
dbfs:/databricks-datasets/power-plant/data/Sheet1.tsv,Sheet1.tsv,308693
dbfs:/databricks-datasets/power-plant/data/Sheet2.tsv,Sheet2.tsv,308693
dbfs:/databricks-datasets/power-plant/data/Sheet3.tsv,Sheet3.tsv,308693
dbfs:/databricks-datasets/power-plant/data/Sheet4.tsv,Sheet4.tsv,308693
dbfs:/databricks-datasets/power-plant/data/Sheet5.tsv,Sheet5.tsv,308693


Ahora, usaremos el comando `dbutils.fs.head` y la función `print` para ver los primeros 65,536 bytes del primer archivo del directorio: `Sheet1.tsv`

In [9]:
#TODO: print the first 65,536 bytes of the file Sheet1.tsv
print(dbutils.fs.head("dbfs:/databricks-datasets/power-plant/data/Sheet1.tsv"))

[Truncated to first 65536 bytes]
AT	V	AP	RH	PE
14.96	41.76	1024.07	73.17	463.26
25.18	62.96	1020.04	59.08	444.37
5.11	39.4	1012.16	92.14	488.56
20.86	57.32	1010.24	76.64	446.48
10.82	37.5	1009.23	96.62	473.9
26.27	59.44	1012.23	58.77	443.67
15.89	43.96	1014.02	75.24	467.35
9.48	44.71	1019.12	66.43	478.42
14.64	45	1021.78	41.25	475.98
11.74	43.56	1015.14	70.72	477.5
17.99	43.72	1008.64	75.04	453.02
20.14	46.93	1014.66	64.22	453.99
24.34	73.5	1011.31	84.15	440.29
25.71	58.59	1012.77	61.83	451.28
26.19	69.34	1009.48	87.59	433.99
21.42	43.79	1015.76	43.08	462.19
18.21	45	1022.86	48.84	467.54
11.04	41.74	1022.6	77.51	477.2
14.45	52.75	1023.97	63.59	459.85
13.97	38.47	1015.15	55.28	464.3
17.76	42.42	1009.09	66.26	468.27
5.41	40.07	1019.16	64.77	495.24
7.76	42.28	1008.52	83.31	483.8
27.23	63.9	1014.3	47.19	443.61
27.36	48.6	1003.18	54.93	436.06
27.47	70.72	1009.97	74.62	443.25
14.6	39.31	1011.11	72.52	464.16
7.91	39.96	1023.57	88.44	475.52
5.81	35.79	1012.14	92.28	484.41
30.53	65.18	1012.69	41.85	437.89
23.87	63.94	1019.02	44.28	445.11
26.09	58.41	1013.64	64.58	438.86
29.27	66.85	1011.11	63.25	440.98
27.38	74.16	1010.08	78.61	436.65
24.81	63.94	1018.76	44.51	444.26
12.75	44.03	1007.29	89.46	465.86
24.66	63.73	1011.4	74.52	444.37
16.38	47.45	1010.08	88.86	450.69
13.91	39.35	1014.69	75.51	469.02
23.18	51.3	1012.04	78.64	448.86
22.47	47.45	1007.62	76.65	447.14
13.39	44.85	1017.24	80.44	469.18
9.28	41.54	1018.33	79.89	482.8
11.82	42.86	1014.12	88.28	476.7
10.27	40.64	1020.63	84.6	474.99
22.92	63.94	1019.28	42.69	444.22
16	37.87	1020.24	78.41	461.33
21.22	43.43	1010.96	61.07	448.06
13.46	44.71	1014.51	50	474.6
9.39	40.11	1029.14	77.29	473.05
31.07	73.5	1010.58	43.66	432.06
12.82	38.62	1018.71	83.8	467.41
32.57	78.92	1011.6	66.47	430.12
8.11	42.18	1014.82	93.09	473.62
13.92	39.39	1012.94	80.52	471.81
23.04	59.43	1010.23	68.99	442.99
27.31	64.44	1014.65	57.27	442.77
5.91	39.33	1010.18	95.53	491.49
25.26	61.08	1013.68	71.72	447.46
27.97	58.84	1002.25	57.88	446.11
26.08	52.3	1007.03	63.34	442.44
29.01	65.71	1013.61	48.07	446.22
12.18	40.1	1016.67	91.87	471.49
13.76	45.87	1008.89	87.27	463.5
25.5	58.79	1016.02	64.4	440.01
28.26	65.34	1014.56	43.4	441.03
21.39	62.96	1019.49	72.24	452.68
7.26	40.69	1020.43	90.22	474.91
10.54	34.03	1018.71	74	478.77
27.71	74.34	998.14	71.85	434.2
23.11	68.3	1017.83	86.62	437.91
7.51	41.01	1024.61	97.41	477.61
26.46	74.67	1016.65	84.44	431.65
29.34	74.34	998.58	81.55	430.57
10.32	42.28	1008.82	75.66	481.09
22.74	61.02	1009.56	79.41	445.56
13.48	39.85	1012.71	58.91	475.74
25.52	69.75	1010.36	90.06	435.12
21.58	67.25	1017.39	79	446.15
27.66	76.86	1001.31	69.47	436.64
26.96	69.45	1013.89	51.47	436.69
12.29	42.18	1016.53	83.13	468.75
15.86	43.02	1012.18	40.33	466.6
13.87	45.08	1024.42	81.69	465.48
24.09	73.68	1014.93	94.55	441.34
20.45	69.45	1012.53	91.81	441.83
15.07	39.3	1019	63.62	464.7
32.72	69.75	1009.6	49.35	437.99
18.23	58.96	1015.55	69.61	459.12
35.56	68.94	1006.56	38.75	429.69
18.36	51.43	1010.57	90.17	459.8
26.35	64.05	1009.81	81.24	433.63
25.92	60.95	1014.62	48.46	442.84
8.01	41.66	1014.49	76.72	485.13
19.63	52.72	1025.09	51.16	459.12
20.02	67.32	1012.05	76.34	445.31
10.08	40.72	1022.7	67.3	480.8
27.23	66.48	1005.23	52.38	432.55
23.37	63.77	1013.42	76.44	443.86
18.74	59.21	1018.3	91.55	449.77
14.81	43.69	1017.19	71.9	470.71
23.1	51.3	1011.93	80.05	452.17
10.72	41.38	1021.6	63.77	478.29
29.46	71.94	1006.96	62.26	428.54
8.1	40.64	1020.66	89.04	478.27
27.29	62.66	1007.63	58.02	439.58
17.1	49.69	1005.53	81.82	457.32
11.49	44.2	1018.79	91.14	475.51
23.69	65.59	1010.85	88.92	439.66
13.51	40.89	1011.03	84.83	471.99
9.64	39.35	1015.1	91.76	479.81
25.65	78.92	1010.83	86.56	434.78
21.59	61.87	1011.18	57.21	446.58
27.98	58.33	1013.92	54.25	437.76
18.8	39.72	1001.24	63.8	459.36
18.28	44.71	1016.99	33.71	462.28
13.55	43.48	1016.08	67.25	464.33
22.99	46.21	1010.71	60.11	444.36
23.94	59.39	1014.32	74.55	438.64
13.74	34.03	1018.69	67.34	470.49
21.3	41.1	1001.86	42.75	455.13
27.54	66.93	1017.06	55.2	450.22
24.81	63.73	1009.

==========================================================================================================================================================================================================
### Ejercicio 2(a)

Ahora usaremos PySpark para visualizar las 5 primeras líneas de los datos

*Hint*: Primero crea un RDD a partir de los datos usando [`sc.textFile("dbfs:/databricks-datasets/power-plant/data")`](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.SparkContext.textFile).

*Hint*: Luego piensa como usar el RDD creado para mostrar datos, el método [`take()`](https://spark.apache.org/docs/1.6.2/api/python/pyspark.html#pyspark.RDD.take) puede ser una buena opción a considerar.

In [11]:
# TODO: Load the data and print the first five lines.
rawTextRdd = sc.textFile("dbfs:/databricks-datasets/power-plant/data")
rawTextRdd.take(5)

Out[ 6 ]: 
[u'AT\tV\tAP\tRH\tPE',
 u'14.96\t41.76\t1024.07\t73.17\t463.26',
 u'25.18\t62.96\t1020.04\t59.08\t444.37',
 u'5.11\t39.4\t1012.16\t92.14\t488.56',
 u'20.86\t57.32\t1010.24\t76.64\t446.48']

A partir nuestra exploración inicial de una muestra de los datos, podemos hacer varias observaciones sobre el proceso de ETL:
- Los datos son un conjunto de .tsv (archivos con valores separados Tab) (es decir, cada fila de datos se separa mediante tabuladores)
- Hay una fila de cabecera, que es el nombre de las columnas
- Parece que el tipo de los datos en cada columna es constante (es decir, cada columna es de tipo double)

El esquema de datos que hemos obtenido de UCI es:
- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output.  Esta es la variable dependiente que queremos predecir usando los otras cuatro

Para usar el paquete Spark CSV [spark-csv](https://spark-packages.org/package/databricks/spark-csv), usaremos el método [sqlContext.read.format()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.format) para especificar el formato de la fuente de datos de entrada: `'com.databricks.spark.csv'`

Podemos especificar diferentes opciones de como importar los datos usando el método [options()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.options). Encontramos las opciones disponible en la documentación de GitHub del paquete [aquí](https://github.com/databricks/spark-csv#features).

Usaremos las siguientes opciones:
- `delimiter='\t'` porque nuestros datos se encuentran delimitados por tabulaciones
- `header='true'` porque nuestro dataset tiene una fila que representa la cabecera de los datos
- `inferschema='true'` porque creemos que todos los datos son números reales, por lo tanto la librería puede inferir el tipo de cada columna de forma automática.

El ultimo componente necesario para crear un DataFrame es determinar la ubicación de los datos usando el método [load()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.load): `"/databricks-datasets/power-plant/data"`

Juntando todo, usaremos la siguiente operación:

`sqlContext.read.format().options().load()`

### Ejercicio 2(b)

Crear un DataFrame a partir de los datos.
- El formato es csv

En el campo opciones incluiremos 3, formadas por nombre de opción y valor, separadas por coma.
- El separador es el tabulador
- El fichero contiene cabecera 'header'
- Para crear un dataframe necesitamos un esquema (schema). A partir de los datos Spark puede tratar de inferir el esquema, le diremos 'true'.

NOTA: [Ayuda] https://docs.databricks.com/spark/latest/data-sources/read-csv.html

El directorio a cargar es el especificado anteriormente. Es importante indicarle a Spark que es una ubicación ya montada en el sistema dbfs, como se ha mostrado en el ejercicio 2a.

In [14]:
# TODO: Replace <FILL_IN> with the appropriate code.
powerPlantDF = sqlContext.read.format('com.databricks.spark.csv').options(delimiter="\t", header="true", inferschema="true").load("dbfs:/databricks-datasets/power-plant/data")

In [15]:
# TEST
from databricks_test_helper import *
expected = set([(s, 'double') for s in ('AP', 'AT', 'PE', 'RH', 'V')])
Test.assertEquals(expected, set(powerPlantDF.dtypes), "Incorrect schema for powerPlantDF")

1 test passed.

Vamos a comprobar los tipos de las columnas usando el metodo [dtypes](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dtypes).

In [17]:
print powerPlantDF.dtypes

[('AT', 'double'), ('V', 'double'), ('AP', 'double'), ('RH', 'double'), ('PE', 'double')]

Tambien podemos examinar los datos usando el metodo `display()`.

In [19]:
display(powerPlantDF)

AT,V,AP,RH,PE
14.96,41.76,1024.07,73.17,463.26
25.18,62.96,1020.04,59.08,444.37
5.11,39.4,1012.16,92.14,488.56
20.86,57.32,1010.24,76.64,446.48
10.82,37.5,1009.23,96.62,473.9
26.27,59.44,1012.23,58.77,443.67
15.89,43.96,1014.02,75.24,467.35
9.48,44.71,1019.12,66.43,478.42
14.64,45.0,1021.78,41.25,475.98
11.74,43.56,1015.14,70.72,477.5


Ahora en lugar de usar [spark-csv](https://spark-packages.org/package/databricks/spark-csv) para inferir los tipos de las columnas, especificaremos el esquema como [DataType](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.DataType), el cual es una lista de [StructField](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.StructType).

La lista completa de tipos se encuentra en el modulo [pyspark.sql.types](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types). Para nuestros datos, usaremos [DoubleType()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.types.DoubleType).

Por ejemplo, para especificar cual es el nombre de la columna usaremos: `StructField(`_name_`,` _type_`, True)`. (El tercer parámetro, `True`, significa que permitimos que la columna tenga valores null.)

### Ejercicio 2(c)

Crea un esquema a medida para el dataset.

In [21]:
# TO DO: Fill in the custom schema.
from pyspark.sql.types import *

# Custom Schema for Power Plant
customSchema = StructType([ \
    StructField('AT', DoubleType(), True), \
    StructField('V' , DoubleType(), True), \
    StructField('AP', DoubleType(), True), \
    StructField('RH', DoubleType(), True), \
    StructField('PE', DoubleType(), True)])

In [22]:
# TEST
Test.assertEquals(set([f.name for f in customSchema.fields]), set(['AT', 'V', 'AP', 'RH', 'PE']), 'Incorrect column names in schema.')
Test.assertEquals(set([f.dataType for f in customSchema.fields]), set([DoubleType(), DoubleType(), DoubleType(), DoubleType(), DoubleType()]), 'Incorrect column types in schema.')

1 test passed.
1 test passed.

### Exercicio 2(d)

Ahora, usaremos el esquema que acabamos de crear para leer los datos. Para realizar esta operación, modificaremos el paso anterior `sqlContext.read.format`. Podemos especificar el esquema haciendo:
- Anadir `schema = customSchema` al método load (simplemente anadelo usando una coma justo después del nombre del archivo)
- Eliminado la opción `inferschema='true'` ya que ahora especificamos el esquema que han de seguir los datos

In [24]:
# TODO: Use the schema you created above to load the data again.
altPowerPlantDF = sqlContext.read.format('com.databricks.spark.csv').options(delimiter="\t", header="true", inferschema="true").load("dbfs:/databricks-datasets/power-plant/data", schema = customSchema)

In [25]:
# TEST
from databricks_test_helper import *
expected = set([(s, 'double') for s in ('AP', 'AT', 'PE', 'RH', 'V')])
Test.assertEquals(expected, set(altPowerPlantDF.dtypes), "Incorrect schema for powerPlantDF")

1 test passed.

Es importante darse cuenta que esta vez no se ha ejecutado ningún job de Spark. Esto se debe a que hemos especificado el esquema, por tanto el paquete [spark-csv](https://spark-packages.org/package/databricks/spark-csv) no tiene por qué leer los datos para inferir el esquema. Podemos usar el método [dtypes](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dtypes) para examinar el nombre y el tipo de los atributos del dataset. Estos deberían ser idénticos a los que hemos inferido anteriormente de los datos.

Cuando ejecutes la siguiente celda, los datos no deberían leerse.

In [27]:
print altPowerPlantDF.dtypes

[('AT', 'double'), ('V', 'double'), ('AP', 'double'), ('RH', 'double'), ('PE', 'double')]

Ahora podemos examinar los datos utilizando el método display(). * Ten en cuenta que esta operación hará que los datos que se lean y se creara el DataFrame. *

In [29]:
display(altPowerPlantDF)

AT,V,AP,RH,PE
14.96,41.76,1024.07,73.17,463.26
25.18,62.96,1020.04,59.08,444.37
5.11,39.4,1012.16,92.14,488.56
20.86,57.32,1010.24,76.64,446.48
10.82,37.5,1009.23,96.62,473.9
26.27,59.44,1012.23,58.77,443.67
15.89,43.96,1014.02,75.24,467.35
9.48,44.71,1019.12,66.43,478.42
14.64,45.0,1021.78,41.25,475.98
11.74,43.56,1015.14,70.72,477.5


## Parte 3: Explorar tus Datos

Ahora que ya hemos cargado los datos, el siguiente paso es explorarlos y realizar algunos análisis y visualizaciones básicas.

Este es un paso que siempre se debe realizar **antes de** intentar ajustar un modelo a los datos, ya que este paso muchas veces nos permitirá conocer una gran información sobre los datos.

En primer lugar vamos a registrar nuestro DataFrame como una tabla de SQL llamado `power_plant`. Debido a que es posible que repitas esta práctica varias veces, vamos a tomar la precaución de eliminar cualquier tabla existente en primer lugar.

Podemos eliminar cualquier tabla SQL existente `power_plant` usando el comando SQL:` DROP TABLE IF EXISTS power_plant` (también debemos que eliminar todos los ficheros asociados a la tabla, lo que podemos hacer con una operación de sistema de archivos Databricks).

Una vez ejecutado el paso anterior, podemos registrar nuestro DataFrame como una tabla de SQL usando [sqlContext.registerDataFrameAsTable()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.registerDataFrameAsTable).

Ejecutar la siguiente celda con el código ya preparado.

In [33]:
sqlContext.sql("DROP TABLE IF EXISTS power_plant")
dbutils.fs.rm("dbfs:/user/hive/warehouse/power_plant", True)
sqlContext.registerDataFrameAsTable(powerPlantDF, "power_plant")

Ahora que nuestro DataFrame existe como una tabla SQL, podemos explorarlo utilizando comandos SQL.

Para ejecutar SQL en una celda, utilizamos el operador `%sql`. La celda siguiente es un ejemplo del uso de SQL para consultar las filas de la tabla de SQL.

**NOTE**: `%sql` es una sentencia que solo funciona en los notebooks de Databricksis. Este ejecuta `sqlContext.sql()` y pasa los resultados a la función `display()`. Estas dos sentencias son equivalentes:

`%sql SELECT * FROM power_plant`

`display(sqlContext.sql("SELECT * FROM power_plant"))`

Ejecutar la siguiente celda con el código ya preparado.

In [36]:
%sql
-- We can use %sql to query the rows
SELECT * FROM power_plant

AT,V,AP,RH,PE
14.96,41.76,1024.07,73.17,463.26
25.18,62.96,1020.04,59.08,444.37
5.11,39.4,1012.16,92.14,488.56
20.86,57.32,1010.24,76.64,446.48
10.82,37.5,1009.23,96.62,473.9
26.27,59.44,1012.23,58.77,443.67
15.89,43.96,1014.02,75.24,467.35
9.48,44.71,1019.12,66.43,478.42
14.64,45.0,1021.78,41.25,475.98
11.74,43.56,1015.14,70.72,477.5


Usa el comando de SQL `desc` para describir el esquema ejecutando la siguiente celda.

In [38]:
%sql
desc power_plant

col_name,data_type,comment
AT,double,null
V,double,null
AP,double,null
RH,double,null
PE,double,null


**Definición de Esquema**

Una vez más, nuestro esquema es el siguiente:

- AT = Atmospheric Temperature in C
- V = Exhaust Vacuum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output

PE es nuestra variable objetivo. Este es el valor que intentamos predecir usando las otras mediciones.

*Referencia [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant)*

Podemos obtener el DataFrame asociado a una tabla SQL usando el método [sqlContext.table()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.table) pasando como argumento el nombre de la tabla SQL.

==========================================================================================================================================================================================================
### Ejercicio 3

Ahora vamos a realizar un análisis estadístico básico de todas las columnas.

Calculad y mostrad los resultados en modo tabla (la función `display` os puede ser de ayuda):
* Número de registros en nuestros datos
* Media de cada columna
* Máximo y mínimo de cada columna
* Desviación estándar de cada columna

Hint: Revisad [DataFrame](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame) ya que contiene métodos que permiten realizar dichos cálculos de manera sencilla.

In [43]:
# Visualización de datos estadísticos básicos de nuestro dataset
df = sqlContext.table("power_plant")
display(df.describe())

summary,AT,V,AP,RH,PE
count,47840,47840,47840,47840,47840
mean,19.651231187290996,54.30580372073594,1013.2590781772572,73.30897784280918,454.36500940635506
stddev,7.452161658340004,12.707361709685806,5.938535418520816,14.599658352081477,17.06628146683769
min,1.81,25.36,992.89,25.56,420.26
max,37.11,81.56,1033.3,100.16,495.76


## <font color="blue">Parte 4: Visualizar los datos</font>

Para entender nuestros datos, intentamos buscar correlaciones entre las diferentes características y sus correspondientes etiquetas. Esto puede ser importante cuando seleccionamos un modelo. Por ejemplo, si una etiqueta y sus características se correlacionan de forma lineal, un modelo de regresión lineal obtendrá un buen rendimiento; por el contrario si la relación es no lineal, modelos más complejos, como arboles de decisión pueden ser una mejor opción. Podemos utilizar herramientas de visualización para observar cada uno de los posibles predictores en relación con la etiqueta como un gráfico de dispersión para ver la correlación entre ellos.

==========================================================================================================================================================================================================
### Ejercicio 4(a)

** Añade las siguientes figuras: **
Vamos a ver si hay una correlación entre la temperatura y la potencia de salida. Podemos utilizar una consulta SQL para crear una nueva tabla que contenga solo el de temperatura (AT) y potencia (PE), y luego usar un gráfico de dispersión con la temperatura en el eje X y la potencia en el eje Y para visualizar la relación (si la hay) entre la temperatura y la energía.

Realiza los siguientes pasos:

- Ejecuta la siguiente celda
- Haz clic en el menú desplegable junto al icono de "Bar Chart" y selecciona "Scatter" para convertir la tabla en un gráfico de dispersión
- Haz click en "Plot Options..."
- En la caja de valores, haz clic en "Temperature" y arrástralo antes de "Power"
- Aplicar los cambios haciendo clic en el botón "Apply"
- Aumentar el tamaño del grafico haciendo clic y arrastrando el control del tamaño

In [45]:
%sql
select AT temperatura, PE potencia from power_plant

temperatura,potencia
14.96,463.26
25.18,444.37
5.11,488.56
20.86,446.48
10.82,473.9
26.27,443.67
15.89,467.35
9.48,478.42
14.64,475.98
11.74,477.5


Parece que hay una gran correlación entre temperatura y power output. Esta correlación es esperable gracias a la segunda ley de la termodinamica [thermal efficiency](https://en.wikipedia.org/wiki/Thermal_efficiency). Ir más allá en este análisis queda fuera del ámbito de esta práctica.

### Ejercicio 4(b)

Usa una sentencia SQL para crear un gráfico de dispersión entre las variables Power (PE) y Exhaust Vacuum Speed (V).

In [48]:
%sql
select AT exhaust_vacuum_speed -- review: no sé traduccion
, PE potencia from power_plant

exhaust_vacuum_speed,potencia
14.96,463.26
25.18,444.37
5.11,488.56
20.86,446.48
10.82,473.9
26.27,443.67
15.89,467.35
9.48,478.42
14.64,475.98
11.74,477.5


Ahora vamos a repetir este ejercicio con el resto de variables y la etiqueta Power Output.

### Ejercicio 4(c)

Usa una sentencia SQL para crear un gráfico de dispersión entre las variables Power (PE) y Pressure (AP).

In [50]:
%sql
select AP presion, PE potencia from power_plant

presion,potencia
1024.07,463.26
1020.04,444.37
1012.16,488.56
1010.24,446.48
1009.23,473.9
1012.23,443.67
1014.02,467.35
1019.12,478.42
1021.78,475.98
1015.14,477.5


### Ejercicio 4(d)

Usa una sentencia SQL para crear un gráfico de dispersión entre las variables Power (PE) y Humidity (RH).

In [52]:
%sql
select RH humedad, PE potencia from power_plant

humedad,potencia
73.17,463.26
59.08,444.37
92.14,488.56
76.64,446.48
96.62,473.9
58.77,443.67
75.24,467.35
66.43,478.42
41.25,475.98
70.72,477.5


##Parte 5: Preparación de los datos

El siguiente paso es preparar los datos para aplicar la regresión. Dado que todo el dataset es numérico y consistente, esta será una tarea sencilla y directa.

El objetivo es utilizar el método de regresión para determinar una función que nos de la potencia de salida como una función de un conjunto de características de predicción. El primer paso en la construcción de nuestra regresión es convertir las características de predicción de nuestro DataFrame a un vector de características utilizando el método [pyspark.ml.feature.VectorAssembler()](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler).

El VectorAssembler es una transformación que combina una lista dada de columnas en un único vector. Esta transformación es muy útil cuando queremos combinar características en crudo de los datos con otras generadas al aplicar diferentes funciones sobre los datos en un único vector de características. Para integrar en un único vector toda esta información antes de ejecutar un algoritmo de aprendizaje automático, el VectorAssembler toma una lista con los nombres de las columnas de entrada (lista de strings) y el nombre de la columna de salida (string).

==========================================================================================================================================================================================================
### Ejercicio 5

- Leer la documentación y los ejemplos de uso de [VectorAssembler](https://spark.apache.org/docs/latest/ml-features.html#vectorassembler)
- Convertir la tabla SQL `power_plant` en un `dataset` llamado datasetDF
- Establecer las columnas de entrada del VectorAssember: `["AT", "V", "AP", "RH"]`
- Establecer la columnas de salida como `"features"`

In [55]:
# TODO: Replace <FILL_IN> with the appropriate code
from pyspark.ml.feature import VectorAssembler

datasetDF = sqlContext.sql("SELECT * FROM power_plant")
input_cols = ["AT", "V", "AP", "RH"]
vectorizer = VectorAssembler()
vectorizer.setInputCols(input_cols)
vectorizer.setOutputCol("features")

Out[ 21 ]: VectorAssembler_4c6ba2520478916d789d

In [56]:
# TEST
Test.assertEquals(set(vectorizer.getInputCols()), {"AT", "V", "AP", "RH"}, "Incorrect vectorizer input columns")
Test.assertEquals(vectorizer.getOutputCol(), "features", "Incorrect vectorizer output column")

1 test passed.
1 test passed.

##Parte 6: Modelar los datos

Ahora vamos a modelar nuestros datos para predecir que potencia de salida se dara cuando tenemos una serie de lecturas de los sensores

La API de [Apache Spark MLlib](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml) ofrece diferentes implementaciones de técnicas de regresion para modelar datasets. En este caso usaremos una muy popular, el llamado [Random Forest](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor). Se basa en la combinación de varios [Arboles de Decisión](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.DecisionTreeRegressor), otra técnica de regresión. En este caso, Random Forest combina una cantidad grande de árboles de decisión independientes en la fase de entrenamiento probados sobre conjuntos de datos aleatorios con igual distribución.
 El objetivo es utilizarse para adaptar un modelo predictivo a un conjunto de datos observados \\(y\\) y \\(X\\). Despues de desarrollar un modelo de este tipo, dado un cierto valor  \\( X\\) del que no conocemos su valor de \\(y \\), el modelo ajustado se puede utilizarse para hacer una prediccion del valor del posible valor \\(y \\). En este caso, queremos predecir la potencia de salida.

NOTA: Animamos a los alumnos a explorar las diferentes técnicas de regresión disponibles en la [API ML de Spark](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.classification)

Necesitamos una forma de evaluar como de bien nuestro modelo predice la produccion de potencia en funcion de parametros de entrada. Podemos hacer esto mediante la division de nuestros datos iniciales establecidos en un _Training set_ utilizado para entrenar a nuestro modelo y un _Test set_ utilizado para evaluar el rendimiento de nuestro modelo. Podemos usar el metodo nativo de los DataFrames [randomSplit()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) para dividir nuestro dataset. El método toma una lista de pesos y una semilla aleatoria opcional. La semilla se utiliza para inicializar el generador de numeros aleatorios utilizado por la funcion de division.

==========================================================================================================================================================================================================
### Ejercicio 6(a)

Utiliza el método [randomSplit()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) para dividir `datasetDF` en trainingSetDF (80% del DataFrame de entrada) y testSetDF (20% del DataFrame de entrada), para poder reproducir siempre el mismo resultado, usar la semilla 1800009193L. Finalmente, cachea (cache()) cada datafrane en memoria para maximizar el rendimiento.

In [59]:
# TODO: Replace <FILL_IN> with the appropriate code.
# We'll hold out 20% of our data for testing and leave 80% for training
seed = 1800009193L
separation_df = [0.2, 0.8]
(split20DF, split80DF) = datasetDF.randomSplit(separation_df, seed)

# Let's cache these datasets for performance
testSetDF = split20DF.cache()
trainingSetDF = split80DF.cache()

In [60]:
# TEST
Test.assertEquals(trainingSetDF.count(), 38243, "Incorrect size for training data set")
Test.assertEquals(testSetDF.count(), 9597, "Incorrect size for test data set")

1 test passed.
1 test passed.

A continuacion vamos a crear nuestro modelo y utilizar su ayda para entender como entrenarlo. Ver la API de [Random Forest Regression](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor) para mas detalles.

Ejecuta la siguiente celda:

In [62]:
# ************ Decision Tree Regression ********************
from pyspark.ml import Pipeline
#from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# I think the originally provided dt is incorrect
#dt = DecisionTreeRegressor()
#print(dt.explainParams())
rf = RandomForestRegressor(featuresCol='features', labelCol='label', predictionCol='prediction')
print(dt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: -2808853809871465425)
varianceCol: column name for the biased sample variance of prediction. (undefined)

La siguiente celda esta basada en [Spark ML Pipeline API for Random Forest Regression](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.RandomForestRegressor).

Ahora, crearemos el [ML Pipeline](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.Pipeline) (flujo de ejecución) y estableceremos las fases del pipeline como vectorizar y posteriormente aplicar el regresor que hemos definido.

Finalmente, crearemos el modelo entrenándolo con el DataFrame `trainingSetDF`.

### Ejercicio 6(b)

El primer paso es establecer los valores de los parametros:
- Define el nombre de la columna a donde guardaremos la prediccion como "Predicted_PE"
- Define el nombre de la columna que contiene la etiqueta como "PE"
- Definimos el numero de arboles de decisión utilizados
- Definimos el grado de profundidad de la estructura en árbol

In [64]:
## TODO: Replace <FILL_IN> with the appropriate code
# Now we set the parameters for the method
rf.setPredictionCol("Predicted_PE")\
  .setLabelCol("PE")\
  .setMaxDepth(5)\
  .setNumTrees(20)
  
# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[vectorizer, rf])

# Train model.  This also runs the indexer.
rfmodel = pipeline.fit(trainingSetDF)

Ejecuta la celda siguiente para ver detalles acerca del modelo construido.

In [66]:
# DISPLAY DETALLES DEL ARBOL CREADO

print("Nodos: " + str(rfmodel.stages[-1]._java_obj.parent().getNumTrees()))
print("Profundidad: "+ str(rfmodel.stages[-1]._java_obj.parent().getMaxDepth()))  # summary only

print(rfmodel.stages[-1]._java_obj.toDebugString())  # summary only

Nodos: 20
Profundidad: 5
RandomForestRegressionModel (uid=RandomForestRegressor_469291478b6d1fce3d99) with 20 trees
 Tree 0 (weight 1.0):
 If (feature 0 <= 17.685000000000002)
 If (feature 0 <= 11.895)
 If (feature 3 <= 93.655)
 If (feature 2 <= 1017.745)
 If (feature 0 <= 9.114999999999998)
 Predict: 483.79103085554
 Else (feature 0 > 9.114999999999998)
 Predict: 476.7195901639346
 Else (feature 2 > 1017.745)
 If (feature 1 <= 44.13)
 Predict: 480.00012658227854
 Else (feature 1 > 44.13)
 Predict: 475.00975177304963
 Else (feature 3 > 93.655)
 If (feature 0 <= 7.93)
 If (feature 1 <= 40.21)
 Predict: 488.4018803418803
 Else (feature 1 > 40.21)
 Predict: 479.92843866171006
 Else (feature 0 > 7.93)
 If (feature 0 <= 9.114999999999998)
 Predict: 476.28333333333336
 Else (feature 0 > 9.114999999999998)
 Predict: 471.3774433656958
 Else (feature 0 > 11.895)
 If (feature 3 <= 87.065)
 If (feature 1 <= 45.825)
 If (feature 1 <= 41.035)
 Predict: 468.60240095465394
 Else (feature 1 > 41.035)
 Predict: 465.81889775413714
 Else (feature 1 > 45.825)
 If (feature 3 <= 77.455)
 Predict: 461.291
 Else (feature 3 > 77.455)
 Predict: 459.5800817995912
 Else (feature 3 > 87.065)
 If (feature 0 <= 15.594999999999999)
 If (feature 0 <= 13.515)
 Predict: 468.43500723588994
 Else (feature 0 > 13.515)
 Predict: 464.99869440459105
 Else (feature 0 > 15.594999999999999)
 If (feature 0 <= 16.625)
 Predict: 459.41035087719297
 Else (feature 0 > 16.625)
 Predict: 455.51262548262554
 Else (feature 0 > 17.685000000000002)
 If (feature 2 <= 1011.885)
 If (feature 3 <= 73.785)
 If (feature 0 <= 25.275)
 If (feature 2 <= 1002.835)
 Predict: 453.50361581920896
 Else (feature 2 > 1002.835)
 Predict: 445.8238480853735
 Else (feature 0 > 25.275)
 If (feature 1 <= 63.905)
 Predict: 439.9873970345963
 Else (feature 1 > 63.905)
 Predict: 434.0868207465277
 Else (feature 3 > 73.785)
 If (feature 1 <= 65.305)
 If (feature 0 <= 21.335)
 Predict: 451.0886936090225
 Else (feature 0 > 21.335)
 Predict: 443.12500672947516
 Else (feature 1 > 65.305)
 If (feature 0 <= 24.665)
 Predict: 440.07911742133535
 Else (feature 0 > 24.665)
 Predict: 433.7973741794311
 Else (feature 2 > 1011.885)
 If (feature 0 <= 22.765)
 If (feature 0 <= 20.490000000000002)
 If (feature 3 <= 68.11500000000001)
 Predict: 459.42967514124297
 Else (feature 3 > 68.11500000000001)
 Predict: 452.8767765567765
 Else (feature 0 > 20.490000000000002)
 If (feature 3 <= 78.695)
 Predict: 451.2277810650887
 Else (feature 3 > 78.695)
 Predict: 446.1937520391517
 Else (feature 0 > 22.765)
 If (feature 0 <= 26.5)
 If (feature 1 <= 56.61)
 Predict: 450.86686974789916
 Else (feature 1 > 56.61)
 Predict: 443.08597717546365
 Else (feature 0 > 26.5)
 If (feature 3 <= 69.545)
 Predict: 437.31209577464784
 Else (feature 3 > 69.545)
 Predict: 433.7886705202309
 Tree 1 (weight 1.0):
 If (feature 0 <= 17.685000000000002)
 If (feature 1 <= 44.845)
 If (feature 0 <= 11.895)
 If (feature 0 <= 9.114999999999998)
 If (feature 0 <= 6.465)
 Predict: 486.5324316682376
 Else (feature 0 > 6.465)
 Predict: 481.8035792580101
 Else (feature 0 > 9.114999999999998)
 If (feature 3 <= 93.655)
 Predict: 476.4158590006844
 Else (feature 3 > 93.655)
 Predict: 471.82205526770286
 Else (feature 0 > 11.895)
 If (feature 1 <= 41.035)
 If (feature 2 <= 1004.645)
 Predict: 464.95153846153846
 Else (feature 2 > 1004.645)
 Predict: 468.49449741756064
 Else (feature 1 > 41.035)
 If (feature 2 <= 1012.375)
 Predict: 464.6828553770087
 Else (feature 2 > 1012.375)
 Predict: 466.4690067720089
 Else (feature 1 > 44.845)
 If (feature 0 <= 14.114999999999998)
 If (feature 1 <= 48.504999999999995)
 If (feature 0 <= 11.895)
 Predict: 477.3424418604651
 Else (feature 0 > 11.895)
 Predict: 467.5182920110194
 Else (feature 1 > 48.504999999999995)
 If (feature 0 <= 10.945)
 Predict: 471.6325581395349
 Else (feature 0 > 10.945)
 Predict: 465.6673039215687
 Else (feature 0 > 14.114999999999998)
 If (feature 3 <= 77.455)
 If (feature 3 <= 43.364999999999995)
 Predic

Ahora estudiaremos cómo se comportan nuestras predicciones en este modelo. Aplicamos nuestro modelo de regresión lineal para el 20% de los datos que hemos separado del conjunto de datos de entrada. La salida del modelo será una columna de producción de electricidad teórica llamada "Predicted_PE".

- Ejecuta la siguiente celda
- Desplázate por la tabla de resultados y observa como los valores de la columna de salida de corriente (PE) se comparan con los valores correspondientes en la salida de potencia predicha  (Predicted_PE)

In [68]:
# Apply our RF model to the test data and predict power output
# Make predictions.
predictions = rfmodel.transform(testSetDF)
display(predictions)

AT,V,AP,RH,PE,features,Predicted_PE
1.81,39.42,1026.92,76.97,490.55,"List(1, 4, List(), List(1.81, 39.42, 1026.92, 76.97))",482.98127599695226
3.2,41.31,997.67,98.84,489.86,"List(1, 4, List(), List(3.2, 41.31, 997.67, 98.84))",481.86516401241454
3.38,41.31,998.79,97.76,489.11,"List(1, 4, List(), List(3.38, 41.31, 998.79, 97.76))",481.86516401241454
3.4,39.64,1011.1,83.43,459.86,"List(1, 4, List(), List(3.4, 39.64, 1011.1, 83.43))",484.28355538891566
3.51,35.47,1017.53,86.56,489.07,"List(1, 4, List(), List(3.51, 35.47, 1017.53, 86.56))",483.992155412435
3.63,38.44,1016.16,87.38,487.87,"List(1, 4, List(), List(3.63, 38.44, 1016.16, 87.38))",483.992155412435
3.91,35.47,1016.92,86.03,488.67,"List(1, 4, List(), List(3.91, 35.47, 1016.92, 86.03))",483.992155412435
3.94,39.9,1008.06,97.49,488.81,"List(1, 4, List(), List(3.94, 39.9, 1008.06, 97.49))",483.29291764535645
4.0,39.9,1009.64,97.16,490.79,"List(1, 4, List(), List(4.0, 39.9, 1009.64, 97.16))",483.4712926803936
4.15,39.9,1007.62,95.69,489.8,"List(1, 4, List(), List(4.15, 39.9, 1007.62, 95.69))",483.8967664828482


A partir de una inspección visual de las predicciones, podemos ver que están cerca de los valores reales.

Sin embargo, nos gustaría disponer de una medida científica exacta de la bondad del modelo. Para realizar esta medición, podemos utilizar una métrica de evaluación como la [Error cuadrático medio](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE) para validar nuestro modelo.

RSME se define como: \\( RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}\\) donde \\(y_i\\) es el valor observado \\(x_i\\) es el valor predicho

RMSE es una medida muy habitual para calcular las diferencias entre los valores predichos por un modelo o un estimador y los valores realmente observados. Cuanto menor sea el RMSE, mejor será nuestro modelo.

Spark ML Pipeline proporciona diferentes métricas para evaluar modelos de regresión, incluyendo [RegressionEvaluator()](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator).

Después de crear una instancia de [RegressionEvaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator), fijaremos el nombre de la columna objetivo "PE" y  el nombre de la columna de predicción a "Predicted_PE". A continuación, invocaremos el evaluador en las predicciones.

### Ejercicio 6(c)
Ejecuta la celda siguiente y asegúrate que entiendes lo que sucede.

In [70]:
# Now let's compute an evaluation metric for our test dataset
from pyspark.ml.evaluation import RegressionEvaluator

# Create an RMSE evaluator using the label and predicted columns
regEval = RegressionEvaluator(predictionCol="Predicted_PE", labelCol="PE", metricName="rmse")

# Run the evaluator on the DataFrame
rmse = regEval.evaluate(predictions)

print("Root Mean Squared Error: %.2f" % rmse)

Root Mean Squared Error: 4.27

Que resultado de RMSE hemos obtenido?

In [72]:
print("Root Mean Squared Error: %.2f" % rmse)

Root Mean Squared Error: 4.27

Otra medida de evaluación estadística muy útil es el coeficiente de determinación, que se denota \\(R ^ 2 \\) o \\(r ^ 2\\) y pronunciado "R cuadrado". Es un número que indica la proporción de la variación en la variable dependiente que es predecible a partir de las variables independientes y proporciona una medida de lo bien que los resultados observados son replicados por el modelo, basado en la proporción de la variación total de los resultados explicada por el modelo. El coeficiente de determinación va de 0 a 1 (más cerca de 1), y cuanto mayor sea el valor, mejor es nuestro modelo.


Para calcular \\(r^2\\), hemos de ejecutar el evaluador `regEval.metricName: "r2"`

Vamos a calcularlo ejecutando la celda siguiente.

In [74]:
# Now let's compute another evaluation metric for our test dataset
r2 = regEval.evaluate(predictions, {regEval.metricName: "r2"})

print("r2: {0:.2f}".format(r2))

r2: 0.94

Que resultado de \\(r^2\\) hemos obtenido? A partir de dicho parametro, crees que el modelo calculado se ajusta bien a los datos?

**Como obtuvimos 0.94, entiendo sí que el modelo calculado se ajusta bien a los datos (hay que considerar que estamos bastante cerca del valor 1, por lo cual tenemos un buen modelo).**

En general, suponiendo una distribución Gaussiana de errores, un buen modelo tendrá 68% de las predicciones dentro de 1 RMSE y 95% dentro de 2 RMSE del valor real (ver http://statweb.stanford.edu/~susan/courses/s60/split/node60.html).

Vamos a examinar las predicciones y ver si un RMSE como el obtenido cumple este criterio.

Crearemos un nuevo DataFrame usando [selectExpr()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.selectExpr) para generar un conjunto de expresiones SQL, y registrar el DataFrame como una tabla de SQL utilizando [registerTempTable()](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.registerTempTable).

### Ejercicio 6(d)

Ejecuta la celda siguiente y asegúrate que entiendes lo que sucede.

In [77]:
# First we remove the table if it already exists
sqlContext.sql("DROP TABLE IF EXISTS Power_Plant_RMSE_Evaluation")
dbutils.fs.rm("dbfs:/user/hive/warehouse/Power_Plant_RMSE_Evaluation", True)

# Next we calculate the residual error and divide it by the RMSE
predictions.selectExpr("PE", "Predicted_PE", "PE - Predicted_PE Residual_Error", "(PE - Predicted_PE) / {} Within_RSME".format(rmse)).registerTempTable("Power_Plant_RMSE_Evaluation")

Podemos utilizar sentencias SQL para explorar la tabla `Power_Plant_RMSE_Evaluation`. En primer lugar vamos a ver qué datos en la tabla utilizando una sentencia SELECT de SQL.

Completa la siguiente consulta para que devuelva todos los elementos de la tabla `Power_Plant_RMSE_Evaluation`.

In [79]:
%sql
SELECT * from Power_Plant_RMSE_Evaluation

PE,Predicted_PE,Residual_Error,Within_RSME
490.55,482.98127599695226,7.568724003047748,1.772038132440924
489.86,481.86516401241454,7.994835987585475,1.8718021990110452
489.11,481.86516401241454,7.244835987585475,1.6962073961360131
459.86,484.28355538891566,-24.423555388915645,-5.718199192032094
489.07,483.992155412435,5.077844587564982,1.1888574925113622
487.87,483.992155412435,3.8778445875649936,0.9079058079113135
488.67,483.992155412435,4.677844587565005,1.095206930978017
488.81,483.29291764535645,5.517082354643549,1.291694651345269
490.79,483.4712926803936,7.318707319606403,1.7135026254484544
489.8,483.8967664828482,5.903233517151818,1.3821028343594077


Ahora podemos mostrar el RMSE como un histograma.

### Ejercicio 6(e)

Ejecuta los siguientes pasos:

- Ejecuta la siguiente celda
- Haz clic en el menú desplegable junto al icono "Bar chart" y selecciona "Histogram" para convertir la tabla en un histograma
- Haz clic en "Plot Options..."
- En la caja "All Fields:", haz clic "&lt;id&gt;" y arrástralo dentro de la caja "Keys:"
- Cambia el valor "Aggregation" a "COUNT"
- Aplicar los cambios haciendo clic en el botón Aplicar
- Aumentar el tamaño del grafico haciendo clic y arrastrando el control del tamaño

Observa que el histograma muestra claramente que el RMSE se centra alrededor de 0 con la gran mayoría de errores dentro de 2 RMSE.

In [81]:
%sql
-- Now we can display the RMSE as a Histogram
SELECT Within_RSME  from Power_Plant_RMSE_Evaluation

Within_RSME
1.772038132440924
1.8718021990110452
1.6962073961360131
-5.718199192032094
1.1888574925113622
0.9079058079113135
1.095206930978017
1.291694651345269
1.7135026254484544
1.3821028343594077


Usando una instrucción SELECT de SQL un poco más compleja, podemos contar el número de predicciones dentro de + o - 1,0 y + o - 2,0 y luego mostrar los resultados como un gráfico circular.

Ejecuta los siguientes pasos:  
  - Ejecutar la siguiente celda
  - Haz clic en el menú desplegable junto al icono de "Bar chart" y selecciona "Pie" para convertir la tabla en un gráfico de sectores
  - Aumentar el tamaño del grafico haciendo clic y arrastrando el control del tamaño

In [83]:
%sql
SELECT case when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1
            when  Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 else 3
       end RSME_Multiple, COUNT(*) AS count
FROM Power_Plant_RMSE_Evaluation
GROUP BY case when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1  when  Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 else 3 end

RSME_Multiple,count
1,6778
3,409
2,2410


### Conclusiones sobre el modelo
A partir del pie chart, responde:
- ¿Cuantas predicciones estan a 1 RMSE como máximo de los valores reales?

**71% de las predicciones de nuestro data set de test están dentro de 1 RMSE**

---------------

- ¿Y cuantas predicciones estan a 2 RMSE como máximo de los valores reales?

**Para 2 RMSE debemos también considerar los que estaban a 1 RMSE, por lo tanto, 96% (71 + 25).**

##Parte 7: Ajustar y evaluar

Ahora que tenemos un primer modelo bastante bueno vamos a tratar de hacer uno aun mejor ajustando sus parametros. El proceso de ajustar un modelo se conoce como [Model Selection](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.tuning) o [Hyperparameter Tuning](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#module-pyspark.ml.tuning). Spark ML Pipeline hace que el proceso de ajuste sea sencillo.

Spark ML Pipeline soporta la seleccion de modelos usando herramientas herramientas como el [CrossValidator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator), que requiere los siguientes elementos:
- [Estimator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.Estimator): un algoritmo o un pipeline a ajustar
- [Conjunto de ParamMaps](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder): parametros para elegir, tambien conocido como _parameter grid_
- [Evaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator): metrica para medir que tan bien lo hace un modelo sobre los datos de entrenamiento

A un alto nivel, las herramientas de seleccion de modelos, tales como [CrossValidator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator) trabajan de la siguiente manera:

- Se separaran los datos de entrada en dos conjuntos entrenamiento y test.
- Para cada uno de estos pares (entrenamiento, test), hay iterar a traves del conjunto de ParamMaps:
    - Para cada [ParamMap](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder), se ajusta el [Estimador](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.Estimator) usando dichos parametros, se obtiene el modelo ajustado, y se evaluar su rendimiento usando el [Evaluator] (https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator).
    - Seleccionan el mejor modelo producido por el conjunto de parametros.

El [Evaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.Evaluator) puede ser por ejemplo un [RegressionEvaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator) para problemas de regresion. Como ayuda a construir el conjunto de parametros, los usuarios pueden utilizar la utilidad [ParamGridBuilder](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder).

Ten en cuenta que la validacion cruzada sobre una conjunto grande de parametros es costosa.

En el siguiente apartado llevaremos a cabo los siguientes pasos:
- Crear un [CrossValidator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator) utilizando un pipeline y un [RegressionEvaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.RegressionEvaluator) que hemos creado anteriormente, y establecer el numero de pliegues (folds) a 5
- Crear una lista de 10 valores distintos para la profundidad de nuestro 'tree'.
  - Crear una lista de 10 parametros de numero de trees.
- Usar [ParamGridBuilder](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder) para construir un conjunto de parametros con los parametros de profundidad y numero de árboles y anadir dicho conjunto al [CrossValidator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator)
- Ejecutar el [CrossValidator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator) para encontrar los parametros que producen el mejor modelo (es decir, mas bajo RMSE) y devolver el mejor modelo.

In [86]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import random


# Parametros pedidos:
folds = 5
depth = random.sample(range(1, 15), 10)
numTrees = random.sample(range(1, 30), 10)

crossval = CrossValidator(estimator=pipeline, evaluator=regEval, numFolds=folds)

paramGrid = (ParamGridBuilder()
               .addGrid(rf.numTrees, numTrees)
               .addGrid(rf.maxDepth, depth)
               .build())

crossval.setEstimatorParamMaps(paramGrid)

rfModel = crossval.fit(trainingSetDF).bestModel


In [87]:
input_cols_rf = input_cols + ["PE", "Predicted_PE"]
print(input_cols_rf)
predictionsAfterTunning = rfModel.transform(testSetDF).select(input_cols_rf)
rmseRF = regEval.evaluate(predictionsAfterTunning)
r2RF = regEval.evaluate(predictionsAfterTunning, {regEval.metricName: "r2"})

print("RF después del tunning -  Root Mean Squared Error: {0:.2f}".format(rmseRF))
print("RF después del tunning - r2: {0:.2f}".format(r2RF))

['AT', 'V', 'AP', 'RH', 'PE', 'Predicted_PE', 'PE', 'Predicted_PE']
RF después del tunning - Root Mean Squared Error: 1.91
RF después del tunning - r2: 0.99

##Responde las siguientes preguntas
- ¿A partir de los resultados del ejercicio 7, cuál es la profundidad y numero de trees de nuestro modelo que ofrece mejores resultados? ¿Podríamos seguir incrementando sus valores indefinidamente? (podéis repetir el ejercicio variando estos valores en la celda correspondiente y analizando su impacto en el resultado final)

In [89]:
print("NumTrees: " + str(rfmodel.stages[-1]._java_obj.parent().getNumTrees()))
print("Profundidad: "+ str(rfmodel.stages[-1]._java_obj.parent().getMaxDepth()))  # summary only

NumTrees: 20
Profundidad: 5

Sobre la pregunta si podemos seguir incrementando indefinidamente, mí respuesta es que no. Primer punto: la ejecución del tunning ha tardado 1.48 hora, o sea, hay un grande coste en aspectos computacionales. Aparte si observas los parametros que he provido para el crossValidator param grid, podrás encontrar que para el numero de trees, ya estabamos comprobando valores entre 1 - 30 (de manera random), pero seguramente ya se hizo pruebas en el cross validation con tales valores y ni por esto significa que simplemente aumentarle resulta en un modelo mejor.

- ¿Cuál te ha parecido la mayor desventaja que ofrece este método de regresión?

Desde mi punto de vista el tiempo de elegir los parámetros óptimos ha sido un poco costoso. Además, puede pasar *overfitting* y por último voy a prover un comentario personal: utilizamos random forest en mi trabajo para detección de fraude y ha pasado que un aumento en el número de arboles ha resultado en relentizar bastante las predicciones en tiempo real.

##Parte 8: Propuesta de modelo alternativo

En base a lo desarrollado en apartados anteriores, tomad una técnica de [regresión de las que Spark](https://spark.apache.org/docs/latest/ml-classification-regression.html) proporciona y aplicadla al problema (Regresión Lineal, Decision Tree, etc.)
Podéis re-utilizar gran parte del código que ya habéis escrito para realizar el modelado del ejercicio 6 en adelante. Para dicho modelo, calcula RSME y el porcentaje de valores que caen en 1 RMSE y 2 RMSE.

In [93]:
# Voy a intentar con una implementacion de una Decision Tree para entonces comparar con la de Randon Forest

from pyspark.ml.regression import DecisionTreeRegressor


dt = DecisionTreeRegressor()

dt.setLabelCol("PE")\
  .setPredictionCol("Predicted_PE")\
  .setFeaturesCol("features")\
  .setMaxBins(100)

dtPipeline = Pipeline()
dtPipeline.setStages([vectorizer, dt])


crossval.setEstimator(dtPipeline)

max_depth = [2, 3, 4, 5]
paramGrid = (ParamGridBuilder()
               .addGrid(dt.maxDepth, max_depth)
               .build())
crossval.setEstimatorParamMaps(paramGrid)
dtModel = crossval.fit(trainingSetDF).bestModel

In [94]:
#Simplemente para no quedar feo
input_cols_dt = input_cols_rf

predictionsDTAfterTunning = dtModel.transform(testSetDF).select(input_cols_dt)
rmseDT = regEval.evaluate(predictionsDTAfterTunning)
r2DT = regEval.evaluate(predictionsDTAfterTunning, {regEval.metricName: "r2"})

print("DT después del tunning -  Root Mean Squared Error: {0:.2f}".format(rmseDT))
print("DT después del tunning -  r2: {0:.2f}".format(r2DT))

DT después del tunning - Root Mean Squared Error: 4.37
DT después del tunning - r2: 0.94

In [95]:
# Usando la misma tabla
sqlContext.sql("DROP TABLE IF EXISTS Power_Plant_RMSE_Evaluation")
dbutils.fs.rm("dbfs:/user/hive/warehouse/Power_Plant_RMSE_Evaluation", True)


predictionsDTAfterTunning.selectExpr("PE", "Predicted_PE", "PE - Predicted_PE Residual_Error", "(PE - Predicted_PE) / {} Within_RSME".format(rmseDT)).registerTempTable("Power_Plant_RMSE_Evaluation")

####Porcentaje que caen en RSME 1 y 2, respectivamente: 70% y 96% (70: 1 + 26: 2)

In [97]:
%sql
SELECT case when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1
            when  Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 else 3
       end RSME_Multiple, COUNT(*) AS count
FROM Power_Plant_RMSE_Evaluation
GROUP BY case when Within_RSME <= 1.0 AND Within_RSME >= -1.0 then 1  when  Within_RSME <= 2.0 AND Within_RSME >= -2.0 then 2 else 3 end

RSME_Multiple,count
1,6702
3,360
2,2535


##Responde 
- ¿A partir de los resultados obtenidos del ejercicio 8, cual crees que en el mejor modelo para el conjunto de datos (ten en cuenta factores como complejidad, rendimiento, ajuste, etc.)?

Para el conjunto de datos en cuestión, aunque el tiempo para sacar los *hyperparams* haya sido bastante más amplio en el modelo de **Random Forest**, él es el mejor observando los resultado de *R2* y *RMSE*.